# Computer Vision, Lab 08: ROS and ORB-SLAM2

Materials for this lab were sourced from these references:

- https://github.com/appliedAI-Initiative/orb_slam_2_ros
- https://github.com/raulmur/ORB_SLAM2
- http://wiki.ros.org/
- https://arxiv.org/abs/1610.06475
- https://roboticsknowledgebase.com/wiki/state-estimation/orb-slam2-setup/
- https://medium.com/@mhamdaan/implementing-orb-slam-on-ubuntu-18-04-ros-melodic-606e668deffa
- https://medium.com/@j.zijlmans/lsd-slam-vs-orb-slam2-a-literature-based-comparison-20732df431d
- https://medium.com/@j.zijlmans/orb-slam-2052515bd84c

## What is ROS?

ROS (Robot Operating System) is an open-source framework for robotics. It provides services need such as hardware abstraction, low-level device control, implementation of commonly-used functionality, message-passing between processes, and package management. It also provides development tools and libraries for building and running your own application code that executes your algorithms and interacts with the robot hardware.

## Install ROS

ROS is so big it really is almost like an operating system, and the development community follows a regular major release pattern similar to that of an OS.
You can try to build ROS from source on just about any system, but the binary releases for Ubuntu are tied to specific versions. If you're running Ubuntu 20.04,
there is only one choice for a ROS binary installation, ROS Noetic. Follow the instructions in the [ROS Noetic installation manual for Ubuntu](http://wiki.ros.org/noetic/Installation/Ubuntu). If you want to give it a shot on Windows10 (not recommended), use the <link>[ROS installation manual for Windows](http://wiki.ros.org/Installation/Windows). Here are the Ubuntu steps in summary:

1. Set up your computer to accept software from packages.ros.org:
       $ sudo sh -c 'echo "deb http://packages.ros.org/ros/ubuntu $(lsb_release -sc) main" > /etc/apt/sources.list.d/ros-latest.list'
2. Install the ROS repository keys:
       $ sudo apt install curl # if you haven't already installed curl
       $ curl -s https://raw.githubusercontent.com/ros/rosdistro/master/ros.asc | sudo apt-key add -
3. Update the local Ubuntu package list including the new ROS package repository
       $ sudo apt update
4. The "Desktop Full Install" (Recommended) contains everything in the Desktop version plus 2D/3D simulators and 2D/3D perception packages. 
       $ sudo apt install ros-noetic-desktop-full
   If you want to install a specific package directly:
       $ sudo apt install ros-noetic-PACKAGE
   For example:
       $ sudo apt install ros-noetic-slam-gmapping
4. Environment setup: you must source the ROS setup script in every bash terminal you use ROS in:
       $ source /opt/ros/noetic/setup.bash
   Alternatively, it can be convenient to automatically source this script every time a new shell is launched. If you're using bash:
       $ echo "source /opt/ros/noetic/setup.bash" >> ~/.bashrc
       $ source ~/.bashrc
5. Finally, install the OS dependencies for building your own packages:
       $ sudo apt install python3-rosdep python3-rosinstall python3-rosinstall-generator python3-wstool build-essential
       $ sudo apt install python3-rosdep
       $ sudo rosdep init
       $ rosdep update

## Install the catkin build system

It seems catkin (needed for development) is not installed with the desktop distribution. Install it now:

    $ sudo apt-get install ros-noetic-catkin
    $ sudo apt-get install python3-catkin-tools
    $ sudo apt-get install python3-osrf-pycommon


## ORB-SLAM2

ORB-SLAM2 is an implementation of the algorithm described in [the original ORB-SLAM paper](https://arxiv.org/pdf/1610.06475.pdf), written by the authors themselves.

### What is ORB-SLAM

ORB-SLAM is a monocular SLAM algorithm. SLAM means Simultaneous Localization And Mapping, and monocular means that we can perform SLAM using a single RGB video or image sequence from a single camera moving through a (assumed static) scene.

SLAM algorithms are algorithms that simultaneously track the movement of the camera (hand-held or mounted onto a robot, car, or UAV) and create a point cloud map of the surroundings that the camera moves through. They create a map of the surroundings while simultaneously localizing the camera within the map.

Monocular SLAM is really just online incremental SfM. It therefore has the same limitation of SfM, namely scale ambiguity. It cannot tell the difference between small motions within a small world or big motions within a big world. This means that any online monocular SLAM method is going to have the problem that the estimated scale of the scene and thus the percieved scale of the scenery will drift over time. ORB-SLAM, like other mono SLAM methods, attempts to be fix the scale drift problem by trying to detect scenery that it has already passed through (a so-called "loop"). Loop detection and correction makes the algorithm work for large-scale outdoor sceneries, smaller-scale indoor scenes, and transitions between these two.

### Feature-based vs. Direct SLAM

Monocular slam algorithms can be divided into two groups:
 - feature-based methods
 - those that use "direct" methods

*Feature-based* SLAM algorithms like ORB-SLAM take the image sequence, search for certain features (key-points, for instance, corners) and only use these features to estimate the camera location and point cloud. This means that they throw away a lot of possibly valuable information from the image, but this they simplify the whole process, because feature points can generally be detected quickly in a rotation invariant manner.

*Direct* SLAM algorithms like LSD-SLAM do not search images for key-points but instead use the image intensities across the entire image to estimate camera motion and dense reconstruction. This means they use more information from the images and thus tend to be create more detailed maps of the surrounding, but on the other hand, they require more computational resources.

<img src="img/lab08-4.png" width="800"/>

The ORB-SLAM2 code repository contains implementations of additional methods. Besides monocular RGB sequences, it also supports stereo images (two images taken with two cameras at the same time instance) and RGBD images (single RGB images augmented with depth information, usually based on IR-range structured light or time of flight).

The ORB-SLAM algorithm utilizes three threads, a tracking thread, a local mapping thread, and a loop closing thread. Here's an overview:

<img src="img/lab08-1.png" width="800"/>

### Initializing the map

To initialize the map, ORB-SLAM uses either two-frame calibrated SfM using two keyframes with sufficient parallax, or a homography between two keyframes imaging a planar scene. To do this, for every candidate keyframe pair, they compute two geometrical models in parallel, one for $\texttt{H}$ and one for $\texttt{F}$. When the feature point motion criteria are achieved, they choose one of the two models based on a relative score. If the motion criteria are not achieved, they restart initialization using the next frame as a candidate keyframe. If the set of tracked feature points becomes too sparse before a model is selected, the entire process is restarted. Finally, once an initial keyframe pair is selected and motion is estimated, the system performs an initial bundle adjustment on the camera positions and 3D initial triangulated 3D points.

### Tracking

Once initialization is achieved, the tracking thread localizes the camera and decides when to insert a new keyframe. Features are matched with the previous frame, and the pose is optimized using motion-only bundle adjustment. The features extracted are FAST corners. (for resolutions up to 752x480, 1000 corners should be good, while for higher resolutions such as KITTI, which is 1241x376, 2000 corners works better). The feature point detection is done at multiple scale levels (related to each other by a factor of 1.2), and each level is divided into a grid in which up to 5 corners per cell are extracted. These FAST corners are then described using ORB. The initial pose is estimated using a constant velocity motion model. If tracking is lost, the place recognition module kicks in and tries to re-localize the camera. When the feature matching is successful, a co-visibility graph of keyframes is used to get a local map consisting of the previous keyframes that share map points with the current frame, the neighbors of these keyframes, and a reference keyframe sharing the most map points with the current frame. Through re-projection, matches of the local map are searched on the frame, then the camera pose is optimized using these matches. Finally, the thread decides whether a new keyframe needs to be created. New keyframes are inserted very frequently at first to make tracking more robust. A new keyframe is created every time at least 20 frames have passed from the last keyframe or the last global re-localization, and optionally when the frame tracks at least 50 points of which less then 90% are successfully tracked from the reference keyframe.

### Local mapping

The local mapping thread is responsible for integrating the information coming from the tracking thread when a new keyframe is added into the map.
First, the new keyframe is inserted into the covisibility graph, which is a spanning tree linking each keyframe to the other keyframes with the most points in common. The local mapper also creates a bag of visual words representation of the keyframe that can be used for loop closing and relocalization later.

New map points are created by triangulating features ORB from connected keyframes in the covisibility graph. The unmachted ORB features in a keyframe are compared with other unmatched ORB features in other keyframes. A match must fulfill the epipolar constraint to be valid. Candidate matches are triangulated and checked for chirality (positive depth in both frames), and the parallax, reprojection error, and scale consistency are also checked. If these checks survive, the match is projected to other connected keyframes to check if it is also in these.

A new map point first needs to go through a test to increase the likelihood of these map points being valid. The point needs to be found in more than 25% of the frames it is predicted to be visible in, and it must be observed by at least three keyframes.

Next, through local bundle adjustment, the current keyframe, all keyframes connected to it through the co-visibility graph, and all the map points seen by these keyframes are optimized using the keyframes that do see the map points but are not connected to the current keyframe.
Finally keyframes that are redundant are discarded to maintain a minimal co-visibility graph. Keyframes for which 90% of the map points can be seen by three other keyframes at the same scale level are discarded.

### Loop closing

To detect possible loops, the loop closing thread checks the bag of words vectors for the current keyframe and its neighbors in the covisibitlity graph. The minimum similarity among the bag of words vectors is taken as a threshold, and all the keyframes with a bag of words similatrity with the current key frame greater than the threshold and all the keyframes already connected to the current keyframe are removed. Among the remaining candidates, if three consistent consecutive keyframes are found, this serious candidate for a possible loop is considered further.

For such candidate loops, a 7 DOF similarity (metric) transformation is calculated using RANSAC followed by BA followed by a second correspondence search and then another BA. If the similarity between the current keyframe and the possible loop canidate is supported by having enough inliers, the loop is accepted.
The current keyframe pose is adjusted and propagated to its neighbors, and corresponding map points are fused. Finally, a pose graph optimization is performed over the essential graph to take out loop closure errors throughout the graph. This process corrects for scale drift.

## Install ORB-SLAM2

The instructions below follow from [the Applied AI Initiative's tutorial](https://github.com/appliedAI-Initiative/orb_slam_2_ros) or the [ORB-SLAM2 GitHub Repository](https://github.com/raulmur/ORB_SLAM2).

1. Create a ROS workspace:
       $ mkdir -p ws/src
       $ cd ws/src
       $ source /opt/ros/noetic/setup.bash
2. Download ORB-SLAM2:
       $ git clone https://github.com/appliedAI-Initiative/orb_slam_2_ros.git
3. Install ORB-SLAM2:
       $ cd ..
       $ rosdep install --from-paths src --ignore-src -r -y
4. Build using catkin:
       $ catkin build

## Run it on benchmark data

1. Open another terminal to run roscore:
       $ cd ws
       $ source /opt/ros/noetic/setup.bash
       $ roscore
2. In the old terminal, launch ORB-SLAM. We'll run the demo of monocular SLAM with the Intel RealSense R200:
       $ roslaunch orb_slam2_ros orb_slam2_r200_mono.launch

| camera | Mono | Stereo | RGBD |
|:----|:----|:----|:----|
|Intel RealSense r200 | roslaunch orb_slam2_ros orb_slam2_r200_mono.launch | roslaunch orb_slam2_ros orb_slam2_r200_stereo.launch | roslaunch orb_slam2_ros orb_slam2_r200_rgbd.launch |
|Intel RealSense d435 | roslaunch orb_slam2_ros orb_slam2_d435_mono.launch | - | roslaunch orb_slam2_ros orb_slam2_d435_rgbd.launch |
Mynteye S | roslaunch orb_slam2_ros orb_slam2_mynteye_s_mono.launch | roslaunch orb_slam2_ros orb_slam2_mynteye_s_stereo.launch | - |
Stereolabs ZED 2 | - | roslaunch orb_slam2_ros orb_slam2_zed2_stereo.launch | - |

## Run ORB-SLAM2 with your Web camera

This process follows [mhamdaan's Medium.com tutorial on ORB-SLAM2](https://medium.com/@mhamdaan/implementing-orb-slam-on-ubuntu-18-04-ros-melodic-606e668deffa).
The following should work with Noetic:

1. Install the ROS USB camera driver and test run:
       $ sudo apt install ros-melodic-usb-cam
       $ roslaunch usb_cam usb_cam-test.launch
2. Install the camera calibration module:
       $ rosdep install camera_calibration
   Once all the dependencies have been installed, you run can the camera_calibration node by giving it the required parameters. To know about each parameter and how to    use camera calibration, see [the ROS camera calibration tutorial page](http://wiki.ros.org/camera_calibration/Tutorials/MonocularCalibration). (You’ll also need a
   checkerboard to perform the calibration.)
   - size: checkerboard size (number of corners)
   - square: rectangle size, in meters
       $ rosrun camera_calibration cameracalibrator.py --size 8x6 --square 0.108 image:=/camera/image_raw camera:=/camera
3. Convert the .ost file to a .yaml file. To do that, enter the following in the terminal:
       $ rosrun  camera_calibration_parsers convert  <filename>.ost <filename>.yaml 
   Once that is over, a new .yaml file with the specified filename will be created.
4. When you run the usb_cam node, it publishes two important topics that will be subscribed by your orb_slam2_ros node. One is the /camera/image_raw and /camera/camera_info. The latter is the topic that sends your camera parameters to the orb_slam2_ros node. Therefore, you need to make your usb_cam node to publish your .yaml file parameters to that topic. To do so, enter the following commands in your terminal:
       $ roscd usb_cam
       $ cd launch
       $ sudo nano usb_cam-test.launch

   The nano text editor will open up your launch file. Enter the highlighted line of code as shown in the image below:
   <text><param name="camera_info_url" value="file:///home/<your_username>/<your_file_address>/<your_filename>.yaml/></text>

   <img src="img/lab08-2.png" width="800"/>
     
   Save the file and close it. Your camera and its parameters are successfully setup
    
5. Setting orb_slam2_ros node (same as above)
       $ cd ~
       $ mkdir -p orbslam2_ws/src
       $ cd orbslam2_ws/src
       $ git clone https://github.com/appliedAI-Initiative/orb_slam_2_ros.git
       $ cd ..
       $ catkin_make
       $ source devel/setup.bash
6. Create a new .launch file in your orb_slam_2_ros/ros/launch directory and paste the following code in it and save it. After that run <code>catkin_make</code> again in your catkin workspace directory:
    <launch>
      <node name="orb_slam2_mono" pkg="orb_slam2_ros"
          type="orb_slam2_ros_mono" output="screen">
        <param name="publish_pointcloud" type="bool" value="true" />
               <param name="publish_pose" type="bool" value="true" />
               <param name="localize_only" type="bool" value="false" />
               <param name="reset_map" type="bool" value="true" />
        <!-- static parameters -->
               <param name="load_map" type="bool" value="false" />
               <param name="map_file" type="string" value="map.bin" />
               <param name="voc_file" type="string" value="$(find orb_slam2_ros)/orb_slam2/Vocabulary/ORBvoc.txt" />
        <param name="pointcloud_frame_id" type="string" value="map" />
               <param name="camera_frame_id" type="string" value="camera_link" />
               <param name="min_num_kf_in_map" type="int" value="5" />
        <!-- ORB parameters -->
               <param name="/ORBextractor/nFeatures" type="int" value="2000" />
               <param name="/ORBextractor/scaleFactor" type="double" value="1.2" />
               <param name="/ORBextractor/nLevels" type="int" value="8" />
               <param name="/ORBextractor/iniThFAST" type="int" value="20" />
               <param name="/ORBextractor/minThFAST" type="int" value="7" />
        <!-- Camera parameters -->
               <!-- Camera frames per second -->
               <param name="camera_fps" type="int" value="30" />
               <!-- Color order of the images (0: BGR, 1: RGB. It is ignored if images are grayscale) -->
               <param name="camera_rgb_encoding" type="bool" value="true" />
                <!--If the node should wait for a camera_info topic to take the camera calibration data-->
               <param name="load_calibration_from_cam" type="bool" value="true" />
      </node>
    </launch>

7. Creating a node to display images

This is an important step because, when your orb_slam2_ros node runs, it will publish the live image from the camera that contains the currently found key points along with a status text. This will be published on the /debug_image topic. We cannot directly view those images, therefore we will have to create a ros node that can display those images to us.

Create a new python file inside a package and paste the following code into it. Make sure that the package has the cv_bridge dependency. If not then, you must add that dependency to your ROS package.

In [ ]:
#!/usr/bin/env python
import rospy
import cv2
from cv_bridge import CvBridge
from sensor_msgs.msg import Image
bridge = CvBridge()
def callback(data):
    frame = bridge.imgmsg_to_cv2(data, "bgr8")
    cv2.imshow('Video Feed', frame)
    cv2.waitKey(1)
    rospy.loginfo('Image feed received!')
def listener():
    rospy.init_node('vid_rec')
    #first parameter is the topic you want to subcribe sensor_msgs/Image from
    rospy.Subscriber('/orb_slam2_mono/debug_image', Image, callback)
    rospy.spin()
if __name__ == '__main__':
    listener()

Save the file and close it. Don’t forget to make your python script executable by the running the following command:
<code>
    $ chmod +x <your_python_script_name>.py
</code>

Now run catkin_make in your catkin workspace directory and then source it.

8. Running it all together. Make sure your camera is connected to your PC. Run each of the following commands in new terminals.

<code>
    $ roscore
    $ roslaunch usb_cam usb_cam-test.launch
    $ roslaunch orb_slam2_ros <your_launch_filename_you_just_created>.launch
    $ rosrun <your_package_name> <your_python_script_name>.py
    $ rviz
</code>
    
All of these must run without any errors. If there are any errors, then run rqt_graph to see if the nodes are communicating with each other. When the RVIZ window opens up, you can add tf and PointCloud2. You must also specify the correct topic name (/map_points) for RVIZ to subscribe to the PointCloud2 message from.
    
<img src="img/lab08-3.png" width="800"/>
    
Once all of that has been set up, you can now see the point clouds appearing and the TF frame of the camera link move in your RVIZ window. Feel free to explore through the GitHub page which contains the source code to use all other functionality of this amazing package. Thanks to the authors (Raul Mur-Artal, Juan D. Tardos, J. M. M. Montiel and Dorian Galvez-Lopez) who developed the orb_slam2_ros package. Now, it seems like you are all so ready to use this package for your next ROS package.

## Run on a video file from our robot

Try to run on a video stream from (one of the <tt>robot.mp4</tt> video files from previous labs)
using the process described in the [ROS OpenCV video stream tutorial](http://wiki.ros.org/video_stream_opencv).

## The report

As always, turn in a report describing your experiments and results by the following week.